In [ ]:
import pandas as pd
import requests
import json
import datetime

In [ ]:
token = 'e7b7fedd-71d0-48c6-8cc7-749e22ba8e80'

In [ ]:
req = requests.get('https://api.sncf.com/v1/coverage/sncf/disruptions?since=20230302T000000&start_page=0&count=1000&',auth=(token, ''))

In [ ]:
doc = json.loads(req.text)
row = len(doc['disruptions'])
print(f'Nombre de lignes : {row}')

Nombre de lignes : 1000


# Retard

## Liste des retards

In [ ]:
df = pd.DataFrame(doc['disruptions'])
#df.head(5)

## Etat du retard

In [ ]:
df_info = pd.DataFrame(list(df['severity']))
df_info['updated_at'] = df['updated_at']

## Combinaison

In [ ]:
df_fin = df_info.copy()
df_fin = df_fin[df_fin['effect'] == 'SIGNIFICANT_DELAYS']
df_fin = df_fin[['effect','updated_at']]

In [ ]:
def conv_heure(string):
    return int(string[0:2]) + int(string[2:4])/60

In [ ]:
gare_d = []
gare_a = []
cause = []
retard = []
heure_arr_prevue = []
heure_arr_reelle = []
heure_depart = []

for i in df_fin.index:
    df_mod = pd.DataFrame(list(df['impacted_objects'][i]))
    df_mod = pd.DataFrame(list(df_mod['impacted_stops'])[0])
    df_mod = df_mod[(df_mod['amended_arrival_time'].notnull())]
    df_mod = df_mod[(df_mod['base_arrival_time'].notnull())]
    
    retard_val = conv_heure(df_mod.iloc[-1]['amended_arrival_time']) - conv_heure(df_mod.iloc[-1]['base_arrival_time'])

    if retard_val<0:
        retard_val = int(round((retard_val+24)*60,1))
    else:
        retard_val = int(round(retard_val*60,1))
    
    df_mod['gare'] = pd.DataFrame(list(df_mod['stop_point']))['name']

    gare_d.append(df_mod.iloc[0]['gare'])
    gare_a.append(df_mod.iloc[-1]['gare'])
    cause.append(df_mod.iloc[-1]['cause'])
    retard.append(retard_val)
    heure_arr_prevue.append(df_mod.iloc[-1]['base_arrival_time'])
    heure_arr_reelle.append(df_mod.iloc[-1]['amended_arrival_time'])
    heure_depart.append(df_mod.iloc[0]['base_departure_time'])


In [ ]:
def get_day(string):
    string = string[:8]
    return string[6:8]+'-'+string[4:6]+'-'+string[0:4]

In [ ]:
df_fin = df_fin.drop(['effect'], axis=1)

df_fin['Gare (départ)'] = gare_d
df_fin['Gare (arrivée)'] = gare_a
df_fin['updated_at'] = df_fin['updated_at'].apply(get_day)
df_fin["Arrivée (réelle)"] = heure_arr_reelle
df_fin["Arrivée (prévue)"] = heure_arr_prevue
df_fin["Départ (prévu)"] = heure_depart
df_fin['Retard (min)'] = retard
df_fin['Cause'] = cause

df_fin.rename(columns = {'updated_at':'Jour'}, inplace = True)


In [ ]:
def str_tps(str):
    return datetime.datetime.strptime(str, '%H%M%S').time()

df_fin["Arrivée (réelle)"] = df_fin["Arrivée (réelle)"].apply(str_tps)
df_fin["Arrivée (prévue)"] = df_fin["Arrivée (prévue)"].apply(str_tps)
df_fin["Départ (prévu)"] = df_fin["Départ (prévu)"].apply(str_tps)

In [ ]:
df_fin

,Jour,Gare (départ),Gare (arrivée),Arrivée (réelle),Arrivée (prévue),Départ (prévu),Retard (min),Cause
2,02-03-2023,Saint-Dizier,Paris Est,07:58:00,07:53:00,05:33:00,5,
3,02-03-2023,Abbeville,Albert,08:28:00,08:23:00,06:49:00,5,Réutilisation d'un train
5,02-03-2023,Toulouse Matabiau,Muret,08:35:00,08:20:00,08:01:00,15,Réutilisation d'un train
6,02-03-2023,Lyon Perrache,Villefranche-sur-Saône,16:54:00,16:49:00,16:14:00,5,Réutilisation d'un train
7,02-03-2023,Amiens,Abbeville,09:37:00,09:27:00,08:53:00,10,Régulation du trafic
...,...,...,...,...,...,...,...,...
970,02-03-2023,Lille Flandres,Paris Nord,08:04:00,07:44:00,06:42:00,20,Régulation du trafic
977,02-03-2023,Paris Nord,Amiens,18:58:00,18:38:00,17:31:00,20,Mise à quai tardive en gare origine
993,02-03-2023,Tours,Nevers,16:03:00,15:58:00,13:30:00,5,Difficultés lors de la préparation du train
994,02-03-2023,Valence Ville,Marseille - Saint-Charles,12:12:00,12:07:00,09:05:00,5,Régulation du trafic


In [ ]:
df_fin['Retard (min)'].mean()

10.20316027088036

In [ ]:
df_fin.to_csv('delays.csv', sep=',', index=False, header=True)